In [1]:
import numpy as np
import matplotlib.pyplot as pyplot

In [15]:
data = np.random.random([1000]) #coming up with arbitrary data and a predictor 
data = np.reshape(data, (100,10)) #reshape to be 100 rows of 10 cols
predict = np.zeros(100)
for i in range(len(data)):
    predict[i] = float(np.random.random()-.5)
    data[i] /= np.sum(data[i]) + predict[i]


In [ ]:
#drawing : 
'''     network : dense                    matrices                                  rows correspond to input layer size + bias, columns to output layer size
                                                                                     X is a bias, it should be a constant 1. pad it onto the back of output.
        1       2       3                    1->2           2->3
        [a]             [f]                 [1 , 2         [1, 2, 3 
        [b]     [d]     [g]                  3 , 4          4, 5, 6
        [c]     [e]     [h]                  5 , 6          7, 8, 9]
        [X]     [X]     [X]                  7 , 8]
        
        forward propagation                                                         each row of input1 corresponds to a row of output. 
                            [1, 2        [1a + 3b + 5c + 7X ,2a + 4b + 6c + 8X]     each col of input2 corresponds to a col of output
                             3, 4   =                                               [1,input layer + bias] @ [input layer + bias, outputlayer] = [1,ouputlayer]
        [a, b, c, X]    x    5, 6      
                             7, 8]                                      intuitively, each neuron is represented by a column of the connecting matrix
                                                                        with each row within representing an axon from each row in input. 
        
        first make sure vals[i][-1] = 1
        intuitively, vals[i] @ layers[i] = vals[i+1]
        then pad output layer with a 1.

        back propagation
            Error : [e1, e2, e3] = [?-f , ?-g , ?-h]
            Values : [[input1, input2, input3, X] , [d, e, X] , [f, g, h, X]]
            -1 * learning_rate * [derivative_of_output] @ [error] @ transpose([value]) = adjustment
            for layer n, output is values[n+1], input is values[n]

                                   [e1                           [e1 * f, e1 * g, e1 * h
        transpose(error) @ value =  e2    x    [f, g, h]    =     e2 * f, e2 * g, e2 * h
                                    e3]                           e3 * f, e3 * g, e3 * h]

        matrix 2->3 += ^that * learningrate * -1

        for subsequent layers
            values layer backprop from : [d,e]
            values layer backprop to : [a,b,c,x]

        derivative = derivative([d,e])

        calculate new d,e from errors by transposing matrix 2->3 (reversing input->output direction) and multiplying it by error
        specifically, d is represented by the values in row 1 of matrix 2->3 , [1,2,3]. so the value we want is f*1, g*2, h*3. 
        for e we want f*4,g*5,h*6. f,g,and h are substituted for e1,e2,and e3, so the operation that gets us what we want is 

                                                                         [1,4,7  
        D,E,X = [e1*1+e2*2+e3*3 , e1*4+e2*5+e3*6, extra] = [e1,e2,e3] @   2,5,8     =  error * transverse(matrix2->3)
                                                                          3,6,9]

        D,E * derivative = delta

        adjustments = delta * transpose(matrix 1->2) = [D, E] @ [1,3,5,7  = [D*1 + E*2, D*3 + E*4, D*5 + E*6, D*7 + E*8] = 
                                                                 2,4,6,8]
        matrix 1->2 += adjustments * -1
        
'''                                     

In [5]:
class Network:
    #layers is a list of ints. each int represents the number of neurons in that layer.
    #the first layer must be the number of columns in the input data
    #the last represents the number of possible outputs
    def __init__(self, layers, learningrate = .01):
        self.learningrate = learningrate
        self.vals = list()                                                  #the '"neurons". 
        self.layers = list()                                                #the "axons and dentrites".
        self.numlayers = len(layers)                                        #paradoxically , len(self.layers) = self.numlayers - 1
        self.layersizes = layers
        for i in range(len(layers)):
            self.vals.append(np.ones((1,layers[i] +1)))                      #our list of row vectors to serve as inputs to the next layer. +1 to add bias value 
            if i > 0 :
                self.layers.append(np.ones((layers[n-1] + 1,layers[n] )))   #our matrix to propagate forward from layer n-1 to layer n.
        return

    def predict(self,input):
        #forward propagation
                                            #inputs shape should be of shape [1,n]
        np.copyto(vals[0], input)           #copy values into our vals[0]
        vals[-1] = 1                        #store our 1 for bias
        for i in range(1,self.numlayers):
            vals[i] = vals[i-1] @ layers[i-1]
            vals[i] = LeakyRelU(vals[i])  if i < self.numlayers -1 else Softmax(vals[i])
            vals[i][:,-1] = 1
        return np.copy(vals[len(vals)-1][:, : vals.shape[1]])                #just return a copy of the last layer without the bias value

    #each row of input should be a row that can be input into predict - each row of testvals a single value that is the correct one. 
    def train(self, input, testvals, epochs = 1):
        #get predictions on each row of input
        predictions = np.zeros(testvals.shape[0])
        for i in range(epochs):
            totalerror = 0
            for row in input:
                p = predict(row)
                #compare them to testvals to get error
                err = testvals-p
                err = np.sum(np.square(err))*1/len(err)                                   #get mse of errors

                #feed error into backpropagate
                backpropagate(err)
                totalerror+= np.sum(err)/input.shape[0]
            #print mean error and 
            print(totalerror)
        return

    def test(self, input, testvals):
        return

def backpropagate(self, error):
    #for 2nd to last layer
    #for each previous one
    layers[-1] -= self.learningrate * np.transpose(error) @  vals[-1][:vals[-1].shape[1]-1]                   #cut off bias value, update final layer
    for i in range( self.numlayers-2,0, -1 ):
        derivative = LeakyRelUDeriv(vals[i])
        d = error @ np.transpose(self.layers[i])
        d = d[:d.shape[1],]                                                                                   #slice off bias value
        delta = d * derivative                                                                                #element wise product. 
        layers[i-1] += learningrate * -1 * np.transpose(layers[i-1])

#takes a numpy array of values and returns a numpy array of the same length. 
def Softmax(inputs):
    return np.exp(inputs)/np.sum(inputs)

def RelU(input):
    return np.max(input,0)

def LeakyRelU(input): #maybe dont use this? More of a demonstration of what should be done inline
    return input * .01 if input < 0 else input

def LeakyRelUDeriv(input):
    return .01 if input < 0 else 1

In [ ]:
#WIP. using this vid as resource : https://www.youtube.com/watch?v=9RN2Wr8xvro&list=PL-nR3Zo5zPQvaNGqElO9-N-1z-4N94qBi&index=1
#but trying to make it easier to use, more general, commented, and without retarded variable naming conventions